In [8]:
import pandas as pd
from pathlib import Path

import qlib
from qlib.data import D
from qlib.strategy.base import BaseStrategy

In [12]:
# 在这里定义路径
# 当前绝对路径
path_cur = Path.cwd() # D:/github/qlib/playGround/jupyter
# 项目根目录
path_project_root = path_cur.parent.parent # 绝对路径 ./qlib

# 数据路径
path_data = path_project_root / Path('./.qlib/qlib_bin')
qlib.init(default_conf="client", provider_uri=path_data, region=REG_CN)

market = 'csi300'
start = '2020-01-01'
end = '2023-03-31'

mkt_csi300 = D.instruments(market='csi300')   # {'market': 'all', 'filter_pipe': []}
fields = ['$open','$close','$high','$low', '$volume', 'Ref($close, 1)', 'Mean($close, 3)', '$high-$low']

# market_df = D.features(D.instruments(market), start_time=start_time, end_time=end_time, freq='day')
# industry_df = D.features(D.instruments(market), fields=['industry'], start_time=start_time, end_time=end_time, freq='day')


[22032:MainThread](2023-04-10 18:17:09,809) INFO - qlib.Initialization - [config.py:415] - default_conf: client.
[22032:MainThread](2023-04-10 18:17:09,815) INFO - qlib.Initialization - [__init__.py:74] - qlib successfully initialized based on client settings.
[22032:MainThread](2023-04-10 18:17:09,816) INFO - qlib.Initialization - [__init__.py:76] - data_path={'__DEFAULT_FREQ': WindowsPath('D:/github/qlib/.qlib/qlib_bin')}


In [ ]:
data = D.features(mkt_csi300, fields, start_time=start, end_time=end, freq='day')

In [ ]:
class MyStrategy(BaseStrategy):
    def __init__(self):
        super().__init__()
        self.mean_reversion = False
        self.beta_norm = False
        self.local_norm = False
        self.warm_start = False

    def on_data(self, event):
        # 获取行情数据
        market_df = event.market_df
        # 获取行业分类数据
        industry_df = event.industry_df

        # 转换成收益率数据
        return_df = market_df['close'].unstack().pct_change().dropna(how='all', axis=1)

        # 标准化收益率数据
        norm_df = (return_df - return_df.mean()) / return_df.std()

        # 行业因素暴露
        industry_exposure_df = pd.get_dummies(industry_df['industry']).reindex(norm_df.columns, fill_value=0)

        # 将标准化收益率数据和行业因素暴露数据拼接在一起
        features_df = pd.concat([norm_df, industry_exposure_df], axis=1)

        # 将NaN值填充为0
        features_df = features_df.fillna(0)

        # 计算趋势跟随因子
        trend_factor = norm_df.apply(lambda x: np.sign(x) * np.abs(x) ** 0.5)

        # 计算动量因子
        momentum_factor = norm_df.apply(lambda x: x - x.rolling(120).mean())

        # 制定交易信号
        signal_df = pd.DataFrame(0, index=norm_df.index, columns=norm_df.columns)

        # 趋势跟随信号
        trend_signal = trend_factor.apply(lambda x: x > x.mean(), axis=1)
        signal_df[trend_signal] = 1

        # 动量信号
        momentum_signal = momentum_factor.apply(lambda x: x > 0, axis=1)
        signal_df[momentum_signal] = 1

        # 输出交易信号
        return signal_df